In [1]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np
import pandas as pd

In [2]:
# 生成字符串型的属性。
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 生成整数型的属性。
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成实数型的属性。
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [ ]:
def to_one_hot(classes, label):
    num_classes = len(classes)
    # print(num_classes)
    # print("label-----------",label)
    label_arr = np.zeros((num_classes))
    # print("label_arr---------",label_arr)
    label_arr[label] += 1.0
    # print("after change label_arr",label_arr)
    return label_arr

In [3]:
# 原始图片的存储位置
orig_picture = os.getcwd()+'\\image\\test'
# 生成图片的存储位置
gen_picture = os.getcwd()+'\\image'
# 需要的识别类型
classes = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'}

# 制作TFRecords数据
def create_record():
    writer = tf.python_io.TFRecordWriter("test.tfrecords")
    for index, name in enumerate(classes):
        class_path = orig_picture + "/" + name + "/"
        print(class_path)
        for img_name in os.listdir(class_path):
            img_path = class_path + img_name
            img = Image.open(img_path)
            img = img.resize((40, 40))  # 设置需要转换的图片大小
            ###图片灰度化######################################################################
            img=img.convert("L")
            ##############################################################################################
            img_raw = img.tobytes()  # 将图片转化为原生bytes
            example = tf.train.Example(
                features=tf.train.Features(feature={
                    "label":   tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
                }))
            writer.write(example.SerializeToString())
    writer.close()

In [4]:
create_record()

C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/3/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/4/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/7/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/1/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/2/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/9/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/0/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/8/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/10/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-brobot\vision\classify\image\test/6/
C:\Users\Administrator\Desktop\brobotmatch\Python\cchess-br

In [5]:
def read_and_decode(filename):
    # 创建文件队列,不限读取的数量    
    filename_queue = tf.train.string_input_producer([filename])
    # create a reader from file queue    
    reader = tf.TFRecordReader()
    # reader从文件队列中读入一个序列化的样本    
    _, serialized_example = reader.read(filename_queue)
    # get feature from serialized example    
    # 解析符号化的样本    
    features = tf.parse_single_example(
        serialized_example,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'img_raw': tf.FixedLenFeature([], tf.string)
        })
    label = features['label']
    label = tf.cast(label, tf.int32)
    image = features['img_raw']
    image = tf.decode_raw(image, tf.uint8)
    image = tf.reshape(image, [40, 40])
    image = tf.cast(image, tf.float32)
    
    return image, label

In [8]:
train_image, train_label = read_and_decode('test.tfrecords')
print(train_image, train_label)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Tensor("Cast_2:0", shape=(40, 40), dtype=float32) Tensor("Cast_1:0", shape=(), dtype=int32)


In [ ]:
classes_l = len(classes) ##类别数为最大数加1
label = tf.one_hot(train_label,classes_l)
label = tf.cast(label, tf.int32)

In [ ]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
tf.train.start_queue_runners(sess=sess)
# train_image, train_label = sess.run([train_image, train_label])
for i in range(5):
    print(train_image, train_label)
    val, label_p = sess.run([train_image, label])
    
    print(val.shape, label_p)

In [6]:
# #載入數據集
# mnist = input_data.read_data_sets("MNIST_data", one_hot = True) 

#每一個批次的大小
batch_size = 20 

#計算一共有多少批次
# n_batch = mnist.train.num_examples // batch_size 
n_batch = 10

#定義兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
x = tf.placeholder(tf.float32, [None, 40, 40])
y = tf.placeholder(tf.int32, [None, 11 ]) 
keep_prob = tf.placeholder(tf.float32)

#建立一個神經網路

x1 = tf.reshape(x, [-1, 40 * 40 * 1])

W1 = tf.Variable(tf.truncated_normal([1600, 4000], stddev=0.1))
b1 = tf.Variable(tf.zeros([4000]) + 0.1)
L1 = tf.nn.tanh(tf.matmul(x1, W1) + b1) #隱藏層的輸出
L1_drop = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([4000, 2000], stddev=0.1))
b2 = tf.Variable(tf.zeros([2000]) + 0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2) #隱藏層的輸出
L2_drop = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.truncated_normal([2000, 1000], stddev=0.1))
b3 = tf.Variable(tf.zeros([1000]) + 0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop, W3) + b3) #隱藏層的輸出
L3_drop = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.truncated_normal([1000, 11], stddev=0.1))
b4 = tf.Variable(tf.zeros([11]) + 0.1)
prediction = tf.nn.tanh(tf.matmul(L3_drop, W4) + b4)


#代價函數 :
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction))
train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)

#初始化變數 operator
init = tf.global_variables_initializer()

#結果存在一個 boolean 的變數中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 回傳一維張量中最大的值，所在的位置

#求準確率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
batch_size = 1000
min_after_dequeue = 300
capacity = min_after_dequeue + 3 * batch_size
train_images, train_labels = tf.train.shuffle_batch([train_image, train_label],
                                      batch_size=batch_size,
                                      num_threads=4,
                                      capacity=capacity,
                                      min_after_dequeue=min_after_dequeue)

test_images, test_labels  = tf.train.shuffle_batch([train_image, train_label],
                                      batch_size=batch_size,
                                      num_threads=4,
                                      capacity=capacity,
                                      min_after_dequeue=min_after_dequeue)

In [10]:
classes_l = 11
train_labels_onehot = tf.cast(tf.one_hot(train_labels,classes_l,on_value=1,off_value=None,axis=1), tf.int32)
test_labels_onehot  = tf.cast(tf.one_hot(test_labels,classes_l,on_value=1,off_value=None,axis=1) , tf.int32)

In [11]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
tf.train.start_queue_runners(sess=sess)
# train_image, train_label = sess.run([train_image, train_label])
for i in range(1):
#     print(train_image, train_label)
#     val, label_p = sess.run([train_image, label])
#     batch_xs, batch_ys = sess.run([train_images, train_labels_onehot])
    print(sess.run([train_labels, train_labels_onehot]))
    
    
#     print(val.shape, label_p)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 

In [ ]:
#每一個批次的大小
batch_size = 10

#計算一共有多少批次
n_batch = 600*11 // batch_size 


sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
tf.train.start_queue_runners(sess=sess)

for epoch in range(4000):
    print(epoch)
    for batch in range(n_batch):

        batch_xs, _, batch_ys = sess.run([train_images, train_labels, train_labels_onehot])
#         print(_, batch_ys)
        feed_dict = {x: batch_xs, y: batch_ys, keep_prob:1.0}
        sess.run(train_step, feed_dict)
          
    batch_xs, _, batch_ys = sess.run([train_images, train_labels, train_labels_onehot])
    feed_dict = {x:batch_xs, y:batch_ys, keep_prob:1.0}
    test_acc = sess.run(accuracy, feed_dict)
    test_xs, _, test_ys = sess.run([test_images, test_labels, test_labels_onehot])
    feed_dict = {x: test_xs, y: test_ys, keep_prob:1.0}  
    train_acc = sess.run(accuracy, feed_dict)
    print("Iter:"+str(epoch)+",test_Acc:"+str(test_acc)+",train_Acc:"+str(train_acc))

0
